In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from include.remote_calls import get_binance_price_coingecko

In [2]:
def intersection(resell_list, sell_list):
    """
    All elements found in resell and not found in sell, thus, active sales.
    """
    lst3 = [value for value in resell_list if value not in sell_list]
    return lst3

In [3]:
trx_df = pd.read_csv("db/transactions/transactions_hist.csv", parse_dates=['date'])

In [4]:
# Check active sales
trx_resell_df = trx_df[trx_df['tr_type'] == 'resell']
trx_resell = trx_resell_df['ship_id'].unique()

trx_sell_df = trx_df[trx_df['tr_type'] == 'sell']
trx_sell = trx_sell_df['ship_id'].unique()

trx_active = intersection(trx_resell, trx_sell)


In [5]:
# Total profit
hist_df = trx_df[~trx_df['ship_id'].isin(trx_active)]
buy_df = hist_df[hist_df['tr_type'] == 'buy']
sell_df = hist_df[hist_df['tr_type'] == 'sell']
bnb_price = get_binance_price_coingecko()

round((sell_df['price'].sum() - buy_df['price'].sum()) * bnb_price,2)

33.15

In [6]:
# Period of time
hist_df['date'].max() - hist_df['date'].min()

Timedelta('56 days 00:00:00')

In [7]:
# [WIP] Active sales. [!!! Get ship stats online not offline]
sold_ships_df = pd.read_csv("db/extracted_sold_BNB_old.csv", header=None)
sold_ships_df.drop_duplicates(subset=[1], keep='last', inplace=True)

ships_active_df = sold_ships_df[sold_ships_df[1].isin(trx_active)][[1,5,6,7,8,9,10]]
ships_active_df.reset_index(drop=True, inplace=True)
columns = ['ship_id','space', 'speed', 'skill', 'defence', 'attack', 'morale']
ships_active_df.columns=columns
ships_active_df['total'] = ships_active_df.drop('ship_id',axis=1).sum(axis=1)
tr_hist_resale_df = trx_df[trx_df['tr_type'] == 'resell']\
    .drop_duplicates(subset=['ship_id'], keep='last')[['ship_id','price']]
ships_active_df.merge(tr_hist_resale_df, on='ship_id')

,ship_id,space,speed,skill,defence,attack,morale,total,price
0,14165,5,15,5,9,14,11,59,0.086
1,18426,7,12,14,12,11,13,69,0.100


In [8]:
# see a total of each stats for all ships sold. bar chart
sold_df = pd.read_csv("db/extracted/all_sold_ships.csv")
# df_agg_sold = sold_df.groupby(['space','speed']).agg('sum')
# df_agg_sold

In [9]:
#